In [12]:
# aircraft_forecasting_optuna.py

# Importaciones básicas
import sys
import os
from pathlib import Path
import logging
import warnings
warnings.filterwarnings('ignore')

# Añadir el directorio padre al path
sys.path.append(str(Path().cwd().parent))

# Importar bibliotecas de análisis
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)

# Importar módulos personalizados
from models import (
    ModelConfig,
    ATCAircraftDataLoader,
    AircraftDataPreprocessor,
    AircraftFeatureEngineer,
    ARIMAModel,
    ProphetModel,
    RandomForestModel,
    LSTMModel,
    EnsembleModel,
    AircraftForecaster
)

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuración
config = ModelConfig()
RANDOM_STATE = 42

In [1]:
# Configuración de almacenamiento para Optuna
import sqlite3
from pathlib import Path

# Crear directorio para almacenamiento si no existe
storage_dir = Path("optuna_storage")
storage_dir.mkdir(exist_ok=True)

# Configurar el almacenamiento
storage_name = f"sqlite:///{storage_dir}/aircraft_forecasting.db"
study_name = "aircraft_forecasting_study"

In [4]:
def load_and_prepare_data(forecast_horizon=7):
    """Carga y prepara los datos para el entrenamiento."""
    logger.info("Cargando y preparando datos...")
    
    # 1. Cargar datos
    data_loader = ATCAircraftDataLoader(config)
    df = data_loader.get_training_data('daily_atc')
    
    # 2. Preprocesar datos
    preprocessor = AircraftDataPreprocessor(config)
    df_processed = preprocessor.preprocess_daily_data(df)
    
    # 3. Ingeniería de características
    feature_engineer = AircraftFeatureEngineer(config)
    df_featured = feature_engineer.create_features(df_processed)
    df_featured = feature_engineer.create_lagged_target(
        df_featured, 
        forecast_horizon=forecast_horizon
    )
    
    # 4. Preparar datos para modelado
    X, y = feature_engineer.select_features_for_model(df_featured)
    
    logger.info(f"Datos preparados: {len(X)} muestras, {len(X.columns)} características")
    
    return X, y, df_featured


In [5]:
def objective(trial, X, y):
    """
    Función objetivo para la optimización con Optuna.
    
    Args:
        trial: Objeto de prueba de Optuna
        X: Características de entrenamiento
        y: Variable objetivo
        
    Returns:
        Error de validación (MAE) a minimizar
    """
    # Selección del modelo
    algorithm = trial.suggest_categorical('algorithm', ['random_forest', 'prophet', 'lstm', 'arima'])

    # Crear una copia de la configuración
    trial_config = ModelConfig()

    if algorithm == 'random_forest':
        # Espacio de búsqueda para Random Forest
        trial_config.models['random_forest'] = {
            'n_estimators': trial.suggest_int('rf_n_estimators', 50, 500, step=50),
            'max_depth': trial.suggest_int('rf_max_depth', 3, 30),
            'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('rf_min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', None]),
            'bootstrap': trial.suggest_categorical('rf_bootstrap', [True, False]),
            'random_state': RANDOM_STATE
        }
        
        model = RandomForestModel(trial_config)
        
    elif algorithm == 'prophet':
        # Espacio de búsqueda para Prophet
        trial_config.models['prophet'] = {
            'yearly_seasonality': trial.suggest_categorical('prophet_yearly', [True, False]),
            'weekly_seasonality': trial.suggest_categorical('prophet_weekly', [True, False]),
            'daily_seasonality': trial.suggest_categorical('prophet_daily', [True, False]),
            'changepoint_prior_scale': trial.suggest_float('prophet_changepoint_prior_scale', 0.001, 0.5, log=True),
            'seasonality_prior_scale': trial.suggest_float('prophet_seasonality_prior_scale', 0.1, 10, log=True),
            'seasonality_mode': trial.suggest_categorical('prophet_seasonality_mode', ['additive', 'multiplicative']),
            'changepoint_range': trial.suggest_float('prophet_changepoint_range', 0.8, 0.95),
            'n_changepoints': trial.suggest_int('prophet_n_changepoints', 10, 50, step=5)
        }
        
        model = ProphetModel(trial_config)
        
    elif algorithm == 'lstm':
        # Espacio de búsqueda para LSTM
        trial_config.models['lstm'] = {
            'sequence_length': trial.suggest_int('lstm_sequence_length', 7, 30, step=7),
            'hidden_units': trial.suggest_int('lstm_hidden_units', 32, 256, step=32),
            'dropout_rate': trial.suggest_float('lstm_dropout', 0.1, 0.5, step=0.1),
            'epochs': trial.suggest_int('lstm_epochs', 50, 200, step=50),
            'batch_size': trial.suggest_categorical('lstm_batch_size', [16, 32, 64]),
            'learning_rate': trial.suggest_float('lstm_learning_rate', 1e-4, 1e-2, log=True),
            'optimizer': trial.suggest_categorical('lstm_optimizer', ['adam', 'rmsprop']),
            'num_layers': trial.suggest_int('lstm_num_layers', 1, 3)
        }
        
        model = LSTMModel(trial_config)
        
    elif algorithm == 'arima':
        # Espacio de búsqueda para ARIMA
        p = trial.suggest_int('arima_p', 0, 5)
        d = trial.suggest_int('arima_d', 0, 2)
        q = trial.suggest_int('arima_q', 0, 5)
        P = trial.suggest_int('arima_P', 0, 3)
        D = trial.suggest_int('arima_D', 0, 2)
        Q = trial.suggest_int('arima_Q', 0, 3)
        s = 7  # Estacionalidad semanal
        
        trial_config.models['arima'] = {
            'order': (p, d, q),
            'seasonal_order': (P, D, Q, s)
        }
        
        model = ARIMAModel(trial_config)
    
    else:
        raise ValueError(f"Tipo de modelo no soportado: {algorithm}")
    
    # Validación cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []
    
    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Ajustar el modelo
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_val)

        # Alinear predicciones con el target y filtrar valores no finitos
        y_pred = np.asarray(y_pred).ravel()

        if len(y_pred) != len(y_val):
            min_len = min(len(y_pred), len(y_val))
            y_pred = y_pred[-min_len:]
            y_val_aligned = y_val.iloc[-min_len:]
        else:
            y_val_aligned = y_val

        valid_mask = np.isfinite(y_pred)
        if not valid_mask.any():
            raise ValueError("Predicciones no válidas: todas son NaN o infinitas")

        y_pred = y_pred[valid_mask]
        y_val_aligned = y_val_aligned.iloc[np.where(valid_mask)[0]]

        # Calcular métricas
        mae = mean_absolute_error(y_val_aligned, y_pred)
        scores.append(mae)
    
    # Devolver el MAE promedio
    return np.mean(scores)


In [6]:
def optimize_hyperparameters(X, y, n_trials=50):
    """
    Optimiza los hiperparámetros usando Optuna.
    
    Args:
        X: Características
        y: Variable objetivo
        n_trials: Número de pruebas a realizar
        
    Returns:
        study: Objeto de estudio de Optuna
    """
    # Crear o cargar estudio con almacenamiento persistente
    try:
        # Intentar cargar un estudio existente
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            load_if_exists=True,
            direction='minimize',
            sampler=TPESampler(seed=RANDOM_STATE)
        )
        logger.info(f"Estudio cargado. Número de trials existentes: {len(study.trials)}")
    except Exception as e:
        # Si no existe, crear uno nuevo
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            direction='minimize',
            sampler=TPESampler(seed=RANDOM_STATE)
        )
        logger.info("Nuevo estudio creado")
    
    # Función objetivo parcial
    def objective_wrapper(trial):
        return objective(trial, X, y)
    
    # Calcular cuántos trials nuevos necesitamos
    remaining_trials = max(0, n_trials - len(study.trials))
    
    if remaining_trials > 0:
        logger.info(f"Iniciando optimización con {remaining_trials} pruebas nuevas...")
        study.optimize(objective_wrapper, n_trials=remaining_trials, show_progress_bar=True)
    else:
        logger.info(f"Ya se han completado {len(study.trials)} trials. No se necesitan más pruebas.")
    
    # Mostrar resultados
    logger.info("\nResumen de la optimización:")
    logger.info(f"Número total de trials: {len(study.trials)}")
    logger.info(f"Mejor valor (MAE): {study.best_value:.4f}")
    logger.info("Mejores parámetros encontrados:")
    for key, value in study.best_params.items():
        logger.info(f"  {key}: {value}")
    
    return study


In [7]:
"""Función principal para ejecutar la optimización."""
# Cargar y preparar datos
X, y, _ = load_and_prepare_data(forecast_horizon=7)

study = optimize_hyperparameters(X, y, n_trials=50)

2026-01-03 13:12:18,456 - __main__ - INFO - Cargando y preparando datos...
2026-01-03 13:12:18,457 - models.data_loader - INFO - Cargando datos diarios ATC desde: data/ATC csvs/atc_dayatcopsummary_202512301506.csv
2026-01-03 13:12:18,474 - models.data_loader - INFO - Datos diarios cargados: 764 registros, columnas: ['arrivals', 'departures', 'overflights', 'nationals', 'unknown', 'total', 'fpp']
2026-01-03 13:12:18,475 - models.data_loader - INFO - Datos de entrenamiento preparados: 764 registros del 2022-12-11 00:00:00 al 2025-12-30 00:00:00
2026-01-03 13:12:18,475 - models.preprocessing - INFO - Iniciando preprocesamiento de datos diarios
2026-01-03 13:12:18,483 - models.preprocessing - INFO - Frecuencia diaria asegurada: 1116 días (original: 764)
2026-01-03 13:12:18,488 - models.preprocessing - INFO - Validación de integridad: OK
2026-01-03 13:12:18,489 - models.preprocessing - INFO - Preprocesamiento completado: 1116 registros
2026-01-03 13:12:18,489 - models.features - INFO - Inic

  0%|          | 0/50 [00:00<?, ?it/s]

2026-01-03 13:12:19,391 - models.model - INFO - Entrenando Prophet
2026-01-03 13:12:19,431 - cmdstanpy - INFO - Chain [1] start processing
2026-01-03 13:12:19,448 - cmdstanpy - INFO - Chain [1] done processing
2026-01-03 13:12:19,448 - cmdstanpy - ERROR - Chain [1] error: code '1' Operation not permitted
2026-01-03 13:12:19,449 - prophet.models - WARNING - Optimization terminated abnormally. Falling back to Newton.
2026-01-03 13:12:19,452 - cmdstanpy - INFO - Chain [1] start processing
2026-01-03 13:12:21,438 - cmdstanpy - INFO - Chain [1] done processing
2026-01-03 13:12:21,439 - models.model - INFO - Prophet entrenado exitosamente
2026-01-03 13:12:21,489 - models.model - INFO - Entrenando Prophet
2026-01-03 13:12:21,505 - cmdstanpy - INFO - Chain [1] start processing
2026-01-03 13:12:21,513 - cmdstanpy - INFO - Chain [1] done processing
2026-01-03 13:12:21,514 - cmdstanpy - ERROR - Chain [1] error: code '1' Operation not permitted
2026-01-03 13:12:21,515 - prophet.models - WARNING - 

[I 2026-01-03 13:12:22,052] Trial 0 finished with value: 1430.0429069998509 and parameters: {'algorithm': 'prophet', 'prophet_yearly': True, 'prophet_weekly': False, 'prophet_daily': False, 'prophet_changepoint_prior_scale': 0.001136467270001117, 'prophet_seasonality_prior_scale': 8.706020878304859, 'prophet_seasonality_mode': 'additive', 'prophet_changepoint_range': 0.8272737450810651, 'prophet_n_changepoints': 15}. Best is trial 0 with value: 1430.0429069998509.


2026-01-03 13:12:22,265 - cmdstanpy - INFO - Chain [1] start processing
2026-01-03 13:12:22,283 - cmdstanpy - INFO - Chain [1] done processing
2026-01-03 13:12:22,285 - models.model - INFO - Prophet entrenado exitosamente
2026-01-03 13:12:22,331 - models.model - INFO - Entrenando Prophet
2026-01-03 13:12:22,348 - cmdstanpy - INFO - Chain [1] start processing
2026-01-03 13:12:22,366 - cmdstanpy - INFO - Chain [1] done processing
2026-01-03 13:12:22,367 - models.model - INFO - Prophet entrenado exitosamente
2026-01-03 13:12:22,411 - models.model - INFO - Entrenando Prophet
2026-01-03 13:12:22,431 - cmdstanpy - INFO - Chain [1] start processing
2026-01-03 13:12:22,458 - cmdstanpy - INFO - Chain [1] done processing
2026-01-03 13:12:22,461 - models.model - INFO - Prophet entrenado exitosamente
2026-01-03 13:12:23.147987: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


...

[I 2026-01-03 13:24:41,298] Trial 47 finished with value: 445.9363904967657 and parameters: {'algorithm': 'arima', 'arima_p': 3, 'arima_d': 1, 'arima_q': 3, 'arima_P': 0, 'arima_D': 1, 'arima_Q': 1}. Best is trial 26 with value: 186.1624744723504.


2026-01-03 13:24:41,510 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:41,524 - models.model - INFO - Entrenando Random Forest con 200 árboles
2026-01-03 13:24:41,681 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:41,696 - models.model - INFO - Entrenando Random Forest con 200 árboles
2026-01-03 13:24:41,883 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:41,896 - models.model - INFO - Entrenando Random Forest con 200 árboles
2026-01-03 13:24:42,122 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:42,137 - models.model - INFO - Entrenando Random Forest con 200 árboles
2026-01-03 13:24:42,398 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:42,480 - models.model - INFO - Entrenando Random Forest con 200 árboles
2026-01-03 13:24:42,602 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:42,616 - models.model - INFO

[I 2026-01-03 13:24:42,424] Trial 48 finished with value: 189.60133581896932 and parameters: {'algorithm': 'random_forest', 'rf_n_estimators': 200, 'rf_max_depth': 27, 'rf_min_samples_split': 5, 'rf_min_samples_leaf': 9, 'rf_max_features': 'sqrt', 'rf_bootstrap': True}. Best is trial 26 with value: 186.1624744723504.


2026-01-03 13:24:42,762 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:42,777 - models.model - INFO - Entrenando Random Forest con 200 árboles
2026-01-03 13:24:42,955 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:42,971 - models.model - INFO - Entrenando Random Forest con 200 árboles
2026-01-03 13:24:43,192 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:43,205 - models.model - INFO - Entrenando Random Forest con 200 árboles
2026-01-03 13:24:43,470 - models.model - INFO - Random Forest entrenado exitosamente
2026-01-03 13:24:43,497 - __main__ - INFO - 
Resumen de la optimización:
2026-01-03 13:24:43,504 - __main__ - INFO - Número total de trials: 50
2026-01-03 13:24:43,506 - __main__ - INFO - Mejor valor (MAE): 186.1625
2026-01-03 13:24:43,506 - __main__ - INFO - Mejores parámetros encontrados:
2026-01-03 13:24:43,507 - __main__ - INFO -   algorithm: random_forest
2026-01-03 13:24:43,508 - __ma

[I 2026-01-03 13:24:43,493] Trial 49 finished with value: 186.1624744723504 and parameters: {'algorithm': 'random_forest', 'rf_n_estimators': 200, 'rf_max_depth': 29, 'rf_min_samples_split': 3, 'rf_min_samples_leaf': 10, 'rf_max_features': 'sqrt', 'rf_bootstrap': True}. Best is trial 26 with value: 186.1624744723504.


In [20]:
def plot_optimization_summary(study):
    """Muestra un resumen visual estilizado de la optimización."""
    fig, axs = plt.subplots(2, 2, figsize=(18, 12), gridspec_kw={"height_ratios": [2, 1.4]})
    fig.suptitle('Resumen de la Optimización de Hiperparámetros', fontsize=20, weight='bold', y=0.98)

    trials_df = study.trials_dataframe().sort_values('number')
    completed = trials_df[trials_df['state'] == 'COMPLETE'].copy()
    best_trial = study.best_trial
    best_algorithm = best_trial.params.get('algorithm', 'N/D')

    # 1. Historial de optimización
    ax_hist = axs[0, 0]
    if not completed.empty:
        completed['mae'] = completed['value'].astype(float)
        completed['best_so_far'] = completed['mae'].cummin()

        sns.lineplot(data=completed, x='number', y='mae', ax=ax_hist, label='MAE por trial', linewidth=2)
        sns.lineplot(data=completed, x='number', y='best_so_far', ax=ax_hist,
                     label='Mejor MAE acumulado', linewidth=2.5, linestyle='--', color='#2ca02c')
        ax_hist.set_title('Historial de Optimización (todos los algoritmos)', weight='bold')
        ax_hist.set_xlabel('Número de trial')
        ax_hist.set_ylabel('MAE')
        ax_hist.set_yscale('log')
        ax_hist.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:.0f}"))
        ax_hist.legend(loc='upper right')
    else:
        ax_hist.text(0.5, 0.5, 'No hay trials completados', ha='center', va='center')

    # 2. Importancia de parámetros
    ax_importance = axs[0, 1]
    best_algo = best_trial.params.get('algorithm')
    
    if best_algo is None:
        ax_importance.text(0.5, 0.5, 'No se pudo identificar el algoritmo ganador', ha='center', va='center')
    else:
        # Usar la función de importancia pasando el eje existente
        plot_hyperparameter_importance(study, ax=ax_importance)
        ax_importance.set_title(f'Importancia de Hiperparámetros\n({best_algo})', weight='bold')

    # 3. Mejores parámetros en formato tabla
    ax_params = axs[1, 0]
    ax_params.axis('off')
    try:
        best_params = study.best_params
        rows = []
        if 'algorithm' in best_params:
            rows.append(['algorithm', f"{best_algorithm}"])
        rows.extend([[k, f"{v}"] for k, v in best_params.items() if k != 'algorithm'])
        table = ax_params.table(cellText=rows, colLabels=['Parámetro', 'Valor'], loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1.2, 1.4)
        ax_params.set_title('Mejores Parámetros', weight='bold', pad=20)
    except Exception:
        ax_params.text(0.5, 0.5, 'No se pudieron obtener los mejores parámetros', ha='center', va='center')

    # 4. Distribución de valores
    ax_dist = axs[1, 1]
    if not completed.empty:
        sns.histplot(data=completed, x='mae', bins=20, kde=True, ax=ax_dist, color='#1f77b4')
        ax_dist.set_title('Distribución del MAE (todos los algoritmos)', weight='bold')
        ax_dist.set_xlabel('MAE')
        ax_dist.set_ylabel('Frecuencia')
        ax_dist.set_yscale('log')
        ax_dist.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.0f}"))
    else:
        ax_dist.text(0.5, 0.5, 'Sin datos suficientes para el histograma', ha='center', va='center')

    fig.text(
        0.5,
        0.015,
        'El historial y la distribución combinan todos los algoritmos evaluados. La tabla y los gráficos de '
        f'importancia/relaciones se centran en el mejor modelo encontrado: {best_algorithm}.',
        ha='center',
        fontsize=11
    )
    plt.tight_layout(rect=[0, 0.05, 1, 0.97])
    return fig

def plot_algorithm_comparison(study):
    """Compara el rendimiento de los algoritmos con métricas claras."""
    trials_df = study.trials_dataframe()
    if 'params_algorithm' not in trials_df.columns:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.text(0.5, 0.5, 'No se registró el parámetro "algorithm" en los trials', ha='center', va='center')
        return fig

    completed = trials_df[trials_df['state'] == 'COMPLETE'].copy()
    completed['mae'] = completed['value'].astype(float)
    completed['algorithm'] = completed['params_algorithm']

    if completed.empty:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.text(0.5, 0.5, 'No hay trials completados para comparar', ha='center', va='center')
        return fig

    summary = (completed.groupby('algorithm')['mae']
               .agg(['median', 'min', 'count'])
               .rename(columns={'median': 'MAE mediano', 'min': 'Mejor MAE', 'count': 'Trials'}))

    fig, axes = plt.subplots(1, 2, figsize=(18, 7), gridspec_kw={'width_ratios': [2, 1]})

    ax_box, ax_bar = axes
    sns.boxplot(data=completed, x='algorithm', y='mae', ax=ax_box, hue='algorithm', palette='viridis', legend=False)
    sns.stripplot(data=completed, x='algorithm', y='mae', ax=ax_box, color='black', size=5, alpha=0.6, jitter=True)
    ax_box.set_title('Distribución del MAE por Algoritmo', weight='bold')
    ax_box.set_xlabel('Algoritmo')
    ax_box.set_ylabel('MAE (escala log)')
    ax_box.set_yscale('log')
    ax_box.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:.0f}"))
    ax_box.axhline(summary['Mejor MAE'].min(), color='red', linestyle='--', linewidth=1.5, label='Mejor MAE global')
    ax_box.legend(loc='upper right')

    summary_sorted = summary.sort_values('Mejor MAE')
    summary_sorted['Algorithm'] = summary_sorted.index
    sns.barplot(data=summary_sorted, y='Algorithm', x='Mejor MAE', ax=ax_bar, palette='mako')
    for i, (mae, count) in enumerate(zip(summary_sorted['Mejor MAE'], summary_sorted['Trials'])):
        ax_bar.text(mae, i, f"{mae:.2f}\n({count} trials)", va='center', ha='left', fontsize=11)
    ax_bar.set_title('Mejor MAE por Algoritmo', weight='bold')
    ax_bar.set_xlabel('Mejor MAE')
    ax_bar.set_ylabel('')
    ax_bar.set_xscale('log')
    ax_bar.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.0f}"))

    plt.tight_layout()
    return fig

def plot_hyperparameter_importance(study, ax=None):
    """Muestra la importancia de los hiperparámetros con anotaciones."""
    create_fig = ax is None
    if create_fig:
        fig, ax = plt.subplots(figsize=(12, 7))
    
    best_trial = study.best_trial
    best_algo = best_trial.params.get('algorithm')

    trials_df = study.trials_dataframe()
    completed = trials_df[trials_df['state'] == 'COMPLETE'].copy()

    if best_algo is None:
        ax.text(0.5, 0.5, 'El parámetro "algorithm" no está disponible en el mejor trial', 
               ha='center', va='center')
        return fig if create_fig else None

    algo_trials = completed[completed['params_algorithm'] == best_algo].copy()
    if len(algo_trials) < 3:
        ax.text(0.5, 0.5, f'No hay suficientes trials para {best_algo}', 
               ha='center', va='center')
        return fig if create_fig else None

    algo_trials['mae'] = algo_trials['value'].astype(float)

    param_cols = [
        col for col in algo_trials.columns
        if col.startswith('params_')
        and col != 'params_algorithm'
        and algo_trials[col].notna().any()
    ]
    
    if not param_cols:
        ax.text(0.5, 0.5, f'{best_algo} no aporta hiperparámetros adicionales', 
               ha='center', va='center')
        return fig if create_fig else None

    def _encode_series(series: pd.Series) -> pd.Series:
        if pd.api.types.is_bool_dtype(series):
            return series.astype(int)
        if pd.api.types.is_numeric_dtype(series):
            return series.astype(float)
        return series.astype(str).astype('category').cat.codes.astype(float)

    importances = {}
    for col in param_cols:
        encoded = _encode_series(algo_trials[col])
        if encoded.dropna().nunique() <= 1:
            continue
        corr = encoded.corr(algo_trials['mae'], method='spearman')
        if pd.notnull(corr):
            importances[col] = abs(corr)

    if not importances:
        ax.text(0.5, 0.5, 
               f'Hiperparámetros de {best_algo} sin variación suficiente para calcular importancia',
               ha='center', va='center')
        return fig if create_fig else None

    importance_df = (pd.DataFrame({
        'Parámetro': [p.replace('params_', '').replace('_', ' ').title() for p in importances.keys()],
        'Importancia': list(importances.values())
    })
    .loc[lambda df: df['Importancia'] > 0]
    .sort_values('Importancia', ascending=True))

    total = importance_df['Importancia'].sum()
    if total == 0:
        ax.text(0.5, 0.5,
               f'No se detectó relación monotónica clara entre los hiperparámetros de {best_algo} y el MAE',
               ha='center', va='center')
        return fig if create_fig else None

    importance_df['Importancia (%)'] = 100 * importance_df['Importancia'] / total

    # Limpiar el eje antes de dibujar
    ax.clear()
    
    # Crear el gráfico de barras con hue para evitar la advertencia
    sns.barplot(data=importance_df, x='Importancia (%)', y='Parámetro', 
                hue='Parámetro', legend=False, ax=ax, palette='crest')
    
    # Añadir las etiquetas de porcentaje
    for i, pct in enumerate(importance_df['Importancia (%)']):
        ax.text(pct + 1, i, f"{pct:.1f}%", va='center', fontsize=11)

    ax.set_title(f'Importancia de Hiperparámetros para {best_algo}', weight='bold')
    ax.set_xlabel('Contribución relativa (Spearman |ρ|)')
    ax.set_ylabel('Parámetro')
    
    if create_fig:
        plt.tight_layout()
        return fig
    return None

def plot_hyperparameter_relationships(study):
    """Explora la relación entre los hiperparámetros numéricos y el MAE."""
    trials_df = study.trials_dataframe()
    completed = trials_df[trials_df['state'] == 'COMPLETE'].copy()

    best_algo = study.best_trial.params.get('algorithm')
    if best_algo is None:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, 'No se pudo identificar el algoritmo ganador', ha='center', va='center')
        return fig

    algo_trials = completed[completed['params_algorithm'] == best_algo].copy()
    if len(algo_trials) < 3:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, f'Se necesitan más trials para {best_algo}', ha='center', va='center')
        return fig

    algo_trials['mae'] = algo_trials['value'].astype(float)

    def _encode_series(series: pd.Series) -> pd.Series:
        if pd.api.types.is_bool_dtype(series):
            return series.astype(int)
        if pd.api.types.is_numeric_dtype(series):
            return series.astype(float)
        return series.astype(str).astype('category').cat.codes.astype(float)

    param_cols = [
        col for col in algo_trials.columns
        if col.startswith('params_')
        and col != 'params_algorithm'
        and algo_trials[col].notna().any()
    ]
    if not param_cols:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, f'{best_algo} no cuenta con hiperparámetros numéricos', ha='center', va='center')
        return fig

    encoded_df = {}
    for col in param_cols:
        encoded = _encode_series(algo_trials[col])
        if encoded.dropna().nunique() <= 1:
            continue
        encoded_df[col] = encoded

    if not encoded_df:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, f'Hiperparámetros de {best_algo} sin variación suficiente', ha='center', va='center')
        return fig

    encoded_df = pd.DataFrame(encoded_df)
    encoded_df['mae'] = algo_trials['mae'].values

    params = list(encoded_df.columns)
    params.remove('mae')
    n_params = len(params)
    n_cols = 2
    n_rows = int(np.ceil(n_params / n_cols))
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows), squeeze=False)

    for idx, param in enumerate(params):
        row, col = divmod(idx, n_cols)
        ax = axes[row][col]
        sns.scatterplot(data=encoded_df, x=param, y='mae', ax=ax, alpha=0.8, s=60)
        sns.regplot(data=encoded_df, x=param, y='mae', scatter=False, ax=ax, color='black', line_kws={'lw': 2})
        ax.set_title(f'{param.replace("params_", "").replace("_", " ").title()} vs MAE', weight='bold')
        ax.set_xlabel(param.replace('params_', '').replace('_', ' ').title())
        ax.set_ylabel('MAE (log)')
        ax.set_yscale('log')
        ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:.0f}"))

    for idx in range(n_params, n_rows * n_cols):
        row, col = divmod(idx, n_cols)
        axes[row][col].axis('off')

    fig.suptitle(f'Relaciones entre Hiperparámetros y MAE para {best_algo}', fontsize=20, weight='bold', y=0.98)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    return fig


In [ ]:
def save_plots(study, output_dir='visualizations'):
    """Guarda todas las visualizaciones en archivos."""
    # Crear directorio si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    # Generar y guardar gráficos
    fig1 = plot_optimization_summary(study)
    fig1.savefig(f'{output_dir}/optimization_summary.png', dpi=300, bbox_inches='tight')
    plt.close(fig1)
    
    fig2 = plot_algorithm_comparison(study)
    fig2.savefig(f'{output_dir}/algorithm_comparison.png', dpi=300, bbox_inches='tight')
    plt.close(fig2)
    
    fig3 = plot_hyperparameter_importance(study)
    fig3.savefig(f'{output_dir}/hyperparameter_importance.png', dpi=300, bbox_inches='tight')
    plt.close(fig3)
    
    try:
        fig4 = plot_hyperparameter_relationships(study)
        fig4.savefig(f'{output_dir}/hyperparameter_relationships.png', dpi=300, bbox_inches='tight')
        plt.close(fig4)
    except Exception as e:
        print(f"No se pudo generar el gráfico de relaciones: {e}")

In [ ]:
# Configuración global de estilo
sns.set_theme(style="whitegrid", context="talk", palette="deep")
plt.rcParams.update(
    {
        "axes.titlesize": 18,
        "axes.labelsize": 14,
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        "legend.fontsize": 11
    }
)